# ╰(*°▽°*)╯ Import stuff ╰(*°▽°*)╯

In [2]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
# import sklearn
# from sklearn import ensemble
# from sklearn.ensemble import RandomForestClassifier
import warnings
import gc


- import MetabolitesFromInChI.csv

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/meidelien/Metabolic-network-layout-using-biochemical-coordinates/main/Notebooks/MetabolitesFromInChIKey.csv")

dfInchiBiGG = pd.read_csv("1515_inchi_only_Groomed.csv")

df1515 = pd.read_csv("https://raw.githubusercontent.com/meidelien/Metabolic-network-layout-using-biochemical-coordinates/main/Notebooks/MetabolitesInChIKeyiML1515.csv")

HTTPError: HTTP Error 404: Not Found

# Creating new df with descriptors
- Iterating over InChI's from MetabolitesFromInChI df 
- Storing DescriptorCalculator output to descrs list
- Creating molDes df & appending Descriptors to each column


In [4]:
mol_list = []

for i in df1515.InChI:
    mol = Chem.MolFromInchi(i)
    mol_list.append(mol)


desc_list=[i[0] for i in Descriptors._descList]
descriptor = MoleculeDescriptors.MolecularDescriptorCalculator(desc_list)

descrs =  []

for i in range(len(mol_list)):
    descrs.append(descriptor.CalcDescriptors(mol_list[i]))
molDes = pd.DataFrame(descrs, columns= desc_list)
molDes.head(5)

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0.000000,0.000000,0.000000,0.000000,0.342643,1.008,0.000,1.007276,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.000000,0.000000,0.000000,0.342643,1.008,0.000,1.007276,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12.382693,-5.621082,12.382693,0.043672,0.069723,663.430,636.214,663.110219,242,0,...,0,0,0,0,0,0,0,0,0,0
3,10.903369,-5.608866,10.903369,0.373579,0.483914,311.187,294.051,311.046597,112,0,...,0,0,0,0,0,0,0,0,0,0
4,9.404861,0.708333,9.404861,0.708333,0.444345,72.107,64.043,72.057515,30,0,...,0,0,0,0,0,0,0,0,0,0


- Make csv cus reasons

In [5]:
molDes.to_csv("chemical_properties1515.csv", index = False)

In [7]:
data1 = pd.read_csv("MetabolitesInChIKeyiML1515.csv")
data2 = pd.read_csv("chemical_properties1515.csv")


- Concatatenating MetabolitesFromInChIKey & chemical_properties.csv

In [8]:
result = pd.concat([data1, data2], axis=1).reindex(data1.index)

In [9]:
result.to_csv("Chemical_descriptors1515.csv", index = False)

In [10]:
df = pd.read_csv("Chemical_descriptors1515.csv")

In [6]:
Species_iD = ["M_h_c", "M_h_c", "M_nadh_c","M_acon_C_c","M_accoa_c","M_6pgl_c", "M_succoa_c", "M_glc__D_e","M_mal__L_c", "M_cit_c", "M_13dpg_c", "M_etoh_c","M_co2_c" ,"M_ru5p__D_c", "M_xu5p__D_c", "M_q8h2_c", "M_akg_c", "M_pyr_c", "M_q8_c", "M_succ_c", "M_e4p_c","M_ac_c","M_2pg_c", "M_actp_c","M_atp_c", "M_gln__L_c", "M_for_c", "M_adp_c","M_oaa_c","M_nadp_c", "M_glx_c", "M_coa_c", "M_3pg_c", "M_amp_c","M_nh4_c", "M_fru_e", "M_pep_c", "M_pi_c", "M_lac__D_c", "M_s7p_c", "M_6pgc_c", "M_etoh_c","M_o2_c", "M_o2_e", "M_acald_c", "M_fum_c", "M_nadph_c", "M_g3p_c", "M_glu__L_c", "M_acald_c", "M_dhap_c", "M_f6p_c", "M_icit_c", "M_pi_c","M_h2o_c"]

In [ ]:
BiGG = ["h", "h", "nadh","acon_C", "accoa", "6pgl", "succoa", "glc__D", "mal__L","cit", "13dpg", "etoh", "co2", "ru5p__D", "xu5p__D", "q8h2", "akg", "pyr", "q8", "succ", "e4p", "ac", "2pg","actp", "atp", "gln__L", "for", "adp", "oaa", "nadp", "glx", "coa"     ]